In [14]:
import pandas as pd
import boto3
import time
import json
import psycopg2
import configparser
from botocore.exceptions import ClientError

In [15]:
# Load DWH Params from a file
config = configparser.ConfigParser()
config.read_file(open("dwh.cfg"))

KEY = config.get("AWS", "KEY")
SECRET = config.get("AWS", "SECRET")

DWH_CLUSTER_TYPE = config.get("DWH", "DWH_CLUSTER_TYPE")
DWH_NUM_NODES = config.get("DWH", "DWH_NUM_NODES")
DWH_NODE_TYPE = config.get("DWH", "DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH", "DWH_CLUSTER_IDENTIFIER")
DWH_DB = config.get("DWH", "DWH_DB")
DWH_DB_USER = config.get("DWH", "DWH_DB_USER")
DWH_DB_PASSWORD = config.get("DWH", "DWH_DB_PASSWORD")
DWH_PORT = config.get("DWH", "DWH_PORT")

DWH_IAM_ROLE_NAME = config.get("DWH", "DWH_IAM_ROLE_NAME")

In [16]:
# Create clients for IAM, EC2, S3 and Redshift
def create_clients():
    ec2 = boto3.resource(
        "ec2",
        region_name="us-west-2",
        aws_access_key_id=KEY,
        aws_secret_access_key=SECRET,
    )

    s3 = boto3.resource(
        "s3",
        region_name="us-west-2",
        aws_access_key_id=KEY,
        aws_secret_access_key=SECRET,
    )

    iam = boto3.client(
        "iam",
        aws_access_key_id=KEY,
        aws_secret_access_key=SECRET,
        region_name="us-west-2",
    )

    redshift = boto3.client(
        "redshift",
        region_name="us-west-2",
        aws_access_key_id=KEY,
        aws_secret_access_key=SECRET,
    )

    return ec2, s3, iam, redshift

In [17]:
def create_role(iam):
    print("Creating role...")
    try:
        print("1.1 Creating a new IAM Role")
        dwhRole = iam.create_role(
            Path="/",
            RoleName=DWH_IAM_ROLE_NAME,
            Description="Allows Redshift clusters to call AWS services on your behalf.",
            AssumeRolePolicyDocument=json.dumps(
                {
                    "Statement": [
                        {
                            "Action": "sts:AssumeRole",
                            "Effect": "Allow",
                            "Principal": {"Service": "redshift.amazonaws.com"},
                        }
                    ],
                    "Version": "2012-10-17",
                }
            ),
        )
    except Exception as e:
        print(e)

    print("1.2 Attaching Policy")

    iam.attach_role_policy(
        RoleName=DWH_IAM_ROLE_NAME,
        PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess",
    )["ResponseMetadata"]["HTTPStatusCode"]

    print("1.3 Get the IAM role ARN")
    roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)["Role"]["Arn"]

    return roleArn

In [18]:
def delete_role(iam):
    print("Deleting role...")
    iam.detach_role_policy(
        RoleName=DWH_IAM_ROLE_NAME,
        PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess",
    )
    iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)

In [19]:
def create_cluster(redshift, roleArn):
    print("Creating cluster...")
    try:
        response = redshift.create_cluster(
            # HW
            ClusterType=DWH_CLUSTER_TYPE,
            NodeType=DWH_NODE_TYPE,
            NumberOfNodes=int(DWH_NUM_NODES),
            # Identifiers & Credentials
            DBName=DWH_DB,
            ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
            MasterUsername=DWH_DB_USER,
            MasterUserPassword=DWH_DB_PASSWORD,
            # Roles (for s3 access)
            IamRoles=[roleArn],
        )
    except Exception as e:
        print(e)

    while (
        redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)[
            "Clusters"
        ][0]["ClusterStatus"]
        != "available"
    ):
        time.sleep(5)

    myClusterProps = redshift.describe_clusters(
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER
    )["Clusters"][0]

    return myClusterProps

In [ ]:
def delete_cluster(redshift):
    print("Deleting cluster...")
    try:
        redshift.delete_cluster(
            ClusterIdentifier=DWH_CLUSTER_IDENTIFIER, SkipFinalClusterSnapshot=True
        )
        print(
            redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)[
                "Clusters"
            ][0]["ClusterStatus"]
        )
    except redshift.exceptions.ClusterNotFoundFault as e:
        print(e)


In [20]:
def _open_tcp_port(ec2, myClusterProps):
    try:
        vpc = ec2.Vpc(id=myClusterProps["VpcId"])
        defaultSg = list(vpc.security_groups.all())[0]
        print(defaultSg)
        defaultSg.authorize_ingress(
            GroupName=defaultSg.group_name,
            CidrIp="0.0.0.0/0",
            IpProtocol="TCP",
            FromPort=int(DWH_PORT),
            ToPort=int(DWH_PORT),
        )
    except ClientError as e:
        code = e.response["Error"]["Code"]
        print(code)

In [21]:
def connect_cluster():

    ec2, s3, iam, redshift = create_clients()
    if iam.get_role(RoleName=DWH_IAM_ROLE_NAME):
        delete_role(iam)
    roleArn = create_role(iam)

    myClusterProps = create_cluster(redshift, roleArn)
    DWH_ENDPOINT = myClusterProps["Endpoint"]["Address"]
    DWH_ROLE_ARN = myClusterProps["IamRoles"][0]["IamRoleArn"]

    if not myClusterProps["PubliclyAccessible"]:
        _open_tcp_port(ec2, myClusterProps)

    print(f"endpoint: {DWH_ENDPOINT}, role_arn: {DWH_ROLE_ARN}")
    print("Cluster is ready!")
    
    return iam, redshift

In [22]:
def clean_up(iam, redshift):
    
    delete_role(iam)
    delete_cluster(redshift)

In [23]:
# Uncomment to connect to cluster

iam, redshift = connect_cluster()

Deleting role...
Creating role...
1.1 Creating a new IAM Role
1.2 Attaching Policy
1.3 Get the IAM role ARN
Creating cluster...
endpoint: dwhcluster.ciy1amcwnctk.us-west-2.redshift.amazonaws.com, role_arn: arn:aws:iam::912348561596:role/dwhRole
Cluster is ready!


In [ ]:
# Copy the above endpoint and role values into the DWH_HOST and ARN fields, respectively in the dfw.cfg file

In [24]:
# Uncomment line below to delete role and cluster

#clean_up(iam, redshift)

Deleting role...


NameError: name 'delete_cluster' is not defined